## Google news api
- This notebooks tests the Google news api
- If time, test unsupervised learning to cluster articles about the same case.  
https://newsapi.org/docs/get-started

In [6]:
import requests
import pandas as pd
import json
from pandas.io.json import json_normalize  
import datetime as dt
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
api_key='7f618b532b6048a78b2a88a6650647af'

### Get the top headlines in Norway(no) now.

In [8]:
params = {
    'apiKey':api_key,
    'country':'no',
    'sources':'',
    'pageSize':100
}
url = 'https://newsapi.org/v2/top-headlines'
response = requests.get(url,params=params)
print(response)

<Response [200]>


In [24]:
df_top = json_normalize(response.json()['articles'])
df_top.head(5)

,author,content,description,publishedAt,source.id,source.name,title,url,urlToImage
0,None,smp-stories-top-widget\r\n– Det var verre enn ...,ISTANBUL (Aftenposten): Israel vil ikke lenger...,2019-02-03T09:45:22Z,aftenposten,Aftenposten,Siste etterlevning fra Oslo-avtalen erklært uø...,https://www.aftenposten.no/verden/i/qn9k9z/Sis...,https://ap.mnocdn.no/images/f0307271-d800-48f3...
1,None,smp-stories-top-widget\r\nEt overveldende fler...,Arbeiderpartiet i Bergen går imot vedtaket om ...,2019-02-03T07:31:36Z,aftenposten,Aftenposten,"Ap i Bergen sier nei til oljeboring i Lofoten,...",https://www.aftenposten.no/norge/i/wElVd5/Ap-i...,https://ap.mnocdn.no/images/7561ad21-84e2-400a...
2,None,smp-stories-top-widget\r\n I et intervju med D...,Professor i økonomisk historie Einar Lie tror ...,2019-02-04T04:14:45Z,aftenposten,Aftenposten,Økonomiprofessor skeptisk til regjeringens olj...,https://www.aftenposten.no/norge/i/Kv5ady/Okon...,https://ap.mnocdn.no/images/30409bba-0676-4fbd...
3,None,smp-stories-top-widget\r\nSaken oppdateres.\r\...,Personen som ble tatt av skredet i Åseral i Ve...,2019-02-03T17:40:06Z,aftenposten,Aftenposten,Gutt (16) døde etter å ha blitt tatt av snøskr...,https://www.aftenposten.no/norge/i/L0raXp/Gutt...,https://ap.mnocdn.no/images/ebf3c834-d270-4165...
4,None,Alt innhold på aftenposten.no + digitale utgav...,Overfylte likhus gir gode tider for Manilas gr...,2019-02-03T18:58:55Z,aftenposten,Aftenposten,"– Graven jeg bygger, kan fort bli min egen",https://www.aftenposten.no/video/i/4dpxeG/-Gra...,https://ap.mnocdn.no/images/9cdcd59e-a1bc-4286...


### Get the all articles in Norway for the last week

In [12]:
today = dt.date.today()
week_ago = today - dt.timedelta(days=2)

In [13]:
params = {
    'apiKey':api_key,
    'language':'',
    'domains':'www.nrk.no,www.vg.no,aftenposten.no,www.db.no',
    'from': week_ago,
    'pageSize':100,
    'page': 1,
    'sortBy':'popularity'
}
url = 'https://newsapi.org/v2/everything'
response = requests.get(url,params=params)
print(response.json()['status'])

ok


In [14]:

pagesize = 100
articles=[]
page = 1
print(page)

params['page']=page
r = requests.get(url,params=params)

while page*pagesize<=r.json()['totalResults'] and r.json()['status'] == 'ok' :
    articles.extend(r.json()['articles'])
    
    page+=1
    params['page']=page
    
    r = requests.get(url,params=params)
    print(params['page'])
    if not r.json()['status'] == 'ok':
        break

1
2
3
4
5
6
7
8
9
10
11


In [15]:
df_last_week = json_normalize(articles)

In [16]:
df_last_week.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
author         719 non-null object
content        987 non-null object
description    1000 non-null object
publishedAt    1000 non-null object
source.id      806 non-null object
source.name    1000 non-null object
title          1000 non-null object
url            1000 non-null object
urlToImage     996 non-null object
dtypes: object(9)
memory usage: 70.4+ KB


In [17]:
df_last_week.describe()

,author,content,description,publishedAt,source.id,source.name,title,url,urlToImage
count,719,987,1000,1000,806,1000,1000,1000,996
unique,169,728,932,860,2,3,951,976,562
top,NRK,"Norsk Presseforbund, logo.\r\nVi arbeider ette...",,2019-02-03T14:54:00Z,nrk,NRK,Finske samer fikk medhold av FN,https://www.vg.no/sport/alpint/i/oRr8KB/aksel-...,https://www.nrk.no/serum/latest/views-dist/typ...
freq,448,193,5,6,614,614,3,2,284


In [239]:
df_last_week.groupby('source.name').describe().head()

author                           content         \
             count unique           top freq   count unique   
source.name                                                   
Aftenposten      0      0           NaN  NaN     208    196   
NRK            596    105           NRK  438     595    346   
Www.vg.no      147     73  Mats Arntzen    7     188    185   

                                                                     \
                                                           top freq   
source.name                                                           
Aftenposten  Alt innhold på aftenposten.no + digitale utgav...    7   
NRK          Norsk Presseforbund, logo.\r\nVi arbeider ette...  194   
Www.vg.no    1 av 2PÅ TUR: Nord-Koreas leder Kim Jong-un vi...    2   

            description        ...                                      title  \
                  count unique ...                                        top   
source.name                    ...                                              
Aftenposten         210    195 ...   Restaurantanmeldelse: Happy av Happolati   
NRK                 596    554 ...           Blir ikke søk i Tamokdalen i dag   
Www.vg.no           194    180 ...      Ny Syria-krangel mellom Tyrkia og USA   

                   url         \
            freq count unique   
source.name                     
Aftenposten    2   210    210   
NRK            3   596    572   
Www.vg.no      2   194    192   

                                                                     \
                                                           top freq   
source.name                                                           
Aftenposten  https://www.aftenposten.no/verden/i/RxMrMx/Bol...    1   
NRK          https://www.nrk.no/nyheter/afd-politiker-alvor...    2   
Www.vg.no    https://www.vg.no/nyheter/innenriks/i/G17MgV/n...    2   

            urlToImage         \
                 count unique   
source.name                     
Aftenposten        209    150   
NRK                596    245   
Www.vg.no          194    167   

                                                                     
                                                           top freq  
source.name                                                          
Aftenposten  http://www.aftenposten.no/public/metaicon-291x...   37  
NRK          https://www.nrk.no/serum/latest/views-dist/typ...  295  
Www.vg.no                 https://vgc.no/hyperion/img/logo.png    6  

[3 rows x 32 columns]

In [23]:
text = df_top.dropna().content.tolist()
text = " ".join(text)
#print(text)

In [22]:
stopwords= set(STOPWORDS)
new_words =open(r'stopwords.txt').read().split()
new_stopwords=stopwords.union(new_words)
wordcloud = WordCloud(stopwords=new_stopwords).generate(text)

width = 15
height = 15
plt.figure(figsize=(width, height))


# Display the generated image:
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

ValueError: We need at least 1 word to plot a word cloud, got 0.